In [1]:
import numpy as np
import pandas as pd

In [2]:
def clean_psp():
    psp = pd.read_csv('price_sales_promotion.csv')
    # Drop those with all zeros
    psp = psp.drop(psp[(psp.Price==0) & (psp.Sales==0) & (psp.Promotions==0)].index)
    # Fix cases where Sales and Price are swapped
    faulty_index = psp[(psp.Sales>psp.Price)].index
    true_price = psp.loc[faulty_index,'Sales']
    true_sales = psp.loc[faulty_index,'Price']
    psp.loc[faulty_index,'Price'] = true_price
    psp.loc[faulty_index,'Sales'] = true_sales
    # Fix cases where Sales is zero and promotions is not
    faulty_index = psp[psp.Sales==0].index
    true_promotions = psp.loc[faulty_index,'Sales']
    true_sales = psp.loc[faulty_index,'Promotions']
    psp.loc[faulty_index,'Promotions'] = true_promotions
    psp.loc[faulty_index,'Sales'] = true_sales
    # Fix cases where Sales and Promotions are swapped
    faulty_index = psp[psp.Promotions/psp.Sales>1].index
    true_promotions = psp.loc[faulty_index,'Sales']
    true_sales = psp.loc[faulty_index,'Promotions']
    psp.loc[faulty_index,'Promotions'] = true_promotions
    psp.loc[faulty_index,'Sales'] = true_sales    
    # Fix individual records
    psp.loc[(psp.SKU=='SKU_27')&(psp.YearMonth==201611),'Price'] =1393.936364
    psp.loc[(psp.SKU=='SKU_27')&(psp.YearMonth==201611),'Sales'] =1393.936364
    psp.loc[(psp.SKU=='SKU_27')&(psp.YearMonth==201612),'Price'] =1916.6625
    psp.loc[(psp.SKU=='SKU_27')&(psp.YearMonth==201612),'Sales'] =1916.6625
    psp.loc[(psp.SKU=='SKU_17')&(psp.YearMonth==201706),'Promotions'] = 184
    psp.loc[(psp.SKU=='SKU_17')&(psp.YearMonth==201706),'Sales'] = 2463.06338-184    
    
    psp.to_csv('price_sales_promotion_cleaned.csv',index=False)

In [3]:
def clean_hv():
    hv = pd.read_csv('historical_volume.csv')
    #Drop those with zeros
    hv = hv.drop(hv[hv.Volume==0].index)
    hv.to_csv('historical_volume_cleaned.csv',index=False)

In [4]:
def read_csvs():
    dfs={}
    dfs['psp'] = pd.read_csv('price_sales_promotion_cleaned.csv')
    dfs['hv'] = pd.read_csv('historical_volume_cleaned.csv')
    dfs['wea']= pd.read_csv('weather.csv')
    dfs['iss'] = pd.read_csv('industry_soda_sales.csv')
    dfs['ec'] = pd.read_csv('event_calendar.csv')
    dfs['iv'] = pd.read_csv('industry_volume.csv')
    dfs['dem'] = pd.read_csv('demographics.csv')
    return dfs

In [5]:
def clean_and_merge():
    clean_psp()
    clean_hv()
    dfs = read_csvs()
    base = dfs['hv'].copy()
    base = base.merge(dfs['psp'],how='left',on=['Agency','SKU','YearMonth'])
    base = base.merge(dfs['wea'],how='left',on=['Agency','YearMonth'])
    base = base.merge(dfs['iss'],how='left',on=['YearMonth'])
    base = base.merge(dfs['ec'],how='left',on=['YearMonth'])
    base = base.merge(dfs['iv'],how='left',on=['YearMonth'])
    base = base.merge(dfs['dem'],how='left',on=['Agency'])    
    base.columns = ['Agency', 'SKU', 'YearMonth', 'Volume', 'Price', 'Sales', 'Promo',
       'Temp', 'SodaVol', 'Easter', 'GoodFriday', 'NewYear',
       'Christmas', 'LaborDay', 'IndDay', 'RevDay','RegGames ', 'FIFA', 'FootballGC',
       'BeerCapital', 'MusicFest', 'IndVol', 'Popu','Income']   
    base = base.drop(['FIFA','FootballGC'],axis=1)
    base.to_csv('integrated_train.csv',index=False)
    
    base = pd.read_csv('test/volume_forecast.csv')
    base['YearMonth']=201801
    base = base.merge(dfs['wea'],how='left',on=['YearMonth','Agency'])
    base = base.merge(dfs['ec'],how='left',on=['YearMonth'])
    base = base.merge(dfs['dem'],how='left',on=['Agency'])
    base = base.drop(['Volume'],axis=1)
    base.columns = ['Agency', 'SKU', 'YearMonth', 'Temp', 'Easter',
           'GoodFriday', 'NewYear', 'Christmas', 'LaborDay', 'IndDay',
           'RevDay', 'RegGames ', 'FIFA',
           'FootballGC', 'BeerCapital', 'MusicFest',
           'Popu', 'Income']
    base.to_csv('integrated_test.csv',index=False)